In [1]:
def path_cost(path : list, weights : dict, cycle=True) -> int:
    """Computes the cost of the given path

    Args:
        path (list): tour taken\n
        weights (dict): key -> edge : value -> weight.\n
        cycle (bool, optional): [description]. Defaults to True.\n

    Returns:
        int: final cost
    """
    cost = 0
    for i in range(len(path) - 1):
        try:
            if (path[i], path[i + 1]) in weights.keys():
                cost += weights[(path[i],path[i+1])]
            else:
                cost += weights[(path[i + 1],path[i])]
        except:
            if cycle:
                if (path[i], path[0]) in weights.keys():
                    cost += weights[(path[i], path[0])]
                else:
                    cost += weights[(path[0], path[i])]
                pass
    return cost


In [2]:
import math
def compute_distance(a, b):
    a = a['pos']
    b = b['pos']
    return math.sqrt(math.pow(a[0]-b[0],2) + math.pow(a[1]-b[1],2))


In [6]:
from graph import MyGraph
import torch
import os
from plotly.subplots import make_subplots
from functools import partial

def create_graph_fittizio(
     name : str,
     dir : str,
     lb : int = 0,
     ub : int = 100,
     n_nodes : int = 50,
     #normalized : bool = False,
     count : int = 0
    ):
    G = nx.Graph()
    V = set([v + 1 for v in range(n_nodes)])
    
    E = set()
    for combination in combinations(V, 2):
        E.add(combination)
    coords = []
    to_normalize_location = []
    max_coord = 0
    for i in range(1,n_nodes + 1):
        location = (random.randint(lb,ub), random.randint(lb,ub))
        if location[0] > max_coord:
            max_coord = location[0]
        elif location[1] > max_coord:
            max_coord = location[1]
        to_normalize_location.append(location)
    #print(to_normalize_location)
    for i, location in enumerate(to_normalize_location):
        normalized_location = (location[0]/max_coord, location[1]/max_coord)
        #print(normalized_location)
        coords.append(normalized_location)
        G.add_node(i+1, pos = normalized_location)
    coords = torch.Tensor(coords)
    for edge in E:
        w = compute_distance(G.nodes[edge[0]], G.nodes[edge[1]])
        #print(edge[0], edge[1], w)
        G.add_edge(edge[0], edge[1], weight = w)
    
    tsp = nx.approximation.traveling_salesman_problem
    path = tsp(G, cycle=True)
    if len(path) > 51:
        return count
    path = np.array(path)
    w_dic = {}
    for i in range(1, 51):
        for j in range(1,51):
            if i != j:
                w_dic[(i,j)] = compute_distance(G.nodes[i], G.nodes[j])
    #print(w_dic)
    cost = path_cost(path, weights = w_dic)
    #print(path, cost)
    

    #TODO salvare i grafi creati
    g = MyGraph(name + '.mio', nodes = G.nodes(), coords = coords,weights = w_dic, sub_opt = path, sub_opt_cost= cost)
    #print(len(path))
    
    path_to_save = os.path.join(dir, name)
    torch.save(g, path_to_save)
    return (count + 1, g)

In [7]:
import networkx as nx
import numpy as np
import random
from itertools import combinations

def get_weight(a, b):
    a = a['pos']
    b = b['pos']
    return int(np.sqrt((a[0]-b[0])**2 + (b[1]-a[1])**2))



In [10]:
from utility import create_dir
import pandas as pd
name_dir = 'plots'
create_dir(name_dir)
count = 0
graphs = []
while count < 10:
    count, g = create_graph_fittizio(f'{name_dir}_' + str(count) + '.mio', name_dir, ub = 1000, count = count)
    graphs.append(g)

In [12]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def get_tour_coords(tour, coords, n_nodes = 50):
    ret = np.zeros((n_nodes+1,2))
    for i, node in enumerate(tour):
        ret[i] = coords[node-1]
    return ret

def create_arrows(coords, color='red'):
    arrows = [go.layout.Annotation(dict(
                x= coords[i+1,0],
                y= coords[i+1,1],
                xref="x", yref="y",
                text="",
                showarrow=True,
                axref = "x", ayref='y',
                ax= coords[i,0],
                ay= coords[i,1],
                arrowhead = 3,
                arrowwidth=1.5,
                arrowcolor=color,)
            ) for i in range(50)]
    return arrows

for g in graphs:   #grafi per cui fare il confronto
    df = pd.DataFrame(g.coords) #coordinate del grafo
    nodes = go.Scatter(
        x=df[0],
        y=df[1],
        mode='markers',
        marker=dict(
            color='rgb(34,163,192)'
                   )
    )

    fig = make_subplots()
    fig.add_trace(nodes)
    fig['layout'].update(height = 600, width = 800, title = 'Example',xaxis=dict(tickangle=-90))

    gt_coords = get_tour_coords(g.get_sub_opt, g.coords) #ottengo un tensore 51,2 di coordinate ordinate in base al tour ottenuto
    fig.update_layout(annotations=create_arrows(gt_coords, color='red'),) #creo tutte le frecce da aggiungere al plot e le aggiungo
    #output_coords = get_tour_coords(output_tour, g.coords) #ripetere lo step però con il tour della rete
    #fig.update_layout(annotations=create_arrows(output_coords),)
    fig.show()

In [8]:
for i in range(10000):
    x = torch.load(f'{name_dir}/{name_dir}_{i}.mio')
    if (len(x.sub_opt)) > 51:
        print(x.sub_opt_cost)
        print(x.coords)


In [9]:
from utility import create_dir
name_dir = 'generated_eval'
create_dir(name_dir)
count = 0
while count < 5000:
    count = create_graph_fittizio(f'{name_dir}_' + str(count) + '.mio', name_dir, ub = 1000, count = count)

Path created correctly
[ 1 18  9 21 36 13 46 25 16  7 33 34 30 40 29  4 15 50 17 19 26 41 44 43
  3 35 42 24 39 47 22 32 14  8 20 23 28 37 31 27  2 49 11 12 45 10  5 48
 38  6  1] 5.441587630984728
[ 1  2 17 35 39  7 21 43  6 23 29 22 34 27 50 44 30 41 15 25 40 31 36 38
 16 10 42 19 24  9 49 46 11 47 32  8 18 12  5  4 28 20 33 26  3 37 13 45
 48 14  1] 5.8189394492657724
[ 1  7 24 21 43 49 25 46 47 29 30  9  5  8 35 48 16 27 44 18 50 23 42 45
 31 13 15 38 37  6 26 12  3  2 10 41 39 17 32 11 28 19 33 22 14 20  4 40
 34 36  1] 6.856547450554646
[ 1 10 32 39  4 50 41 14 29  5 21 20 33 26 44 47 38 23 46  7 13  2 48  6
 24 42 37 36 18  3 27 11  9 30 19 28 25 49 16 31  8 34 43 45 15 40 35 12
 22 17  1] 6.477656722016265
[ 1 19 16 26 30 39  5 34 22 32 42 33 13 31 29  8 10 35 46 37  2 45 15 47
 41 40  4 17 21 43 44 36 24 50 48 12 14 49  3 23 25 18  9  6 11 20 28  7
 27 38  1] 5.838717697773516
[ 1 43 42 11  3 50 12  9 41 45 19 14  4 48 40 13 30  7 26 21 44 35 34 38
  5 36 25 20 29 17 27 18 28 

In [ ]:
for i in range(5000):
    x = torch.load(f'{name_dir}/{name_dir}_{i}.mio')
    if (len(x.sub_opt)) != 51 or (x.sub_opt_cost > 8):
        print(i)
        print(x.coords.shape)
        print(x.coords)
        print(x.nodes)
